In [0]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2020-06-10 06:06:22--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M  12.7MB/s    in 16s     

2020-06-10 06:06:38 (19.0 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



In [0]:
# Ekstrak File
import zipfile,os
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/rockpaperscissors/rps-cv-images'
train_dir = os.path.join(base_dir)
validation_dir = os.path.join(base_dir)

In [0]:
datagen = ImageDataGenerator(validation_split=0.3,rescale=1./255)

In [0]:
# pip install split_folders

In [0]:
# # https://pypi.org/project/split-folders/
# import split_folders
# split_folders.ratio(base_dir,output='/tmp/rockpaperscissors/data',seed=1337, ratio=(.8, .2))

In [7]:
# folder akan di split otomatis di image generator menjadi val dan train
train_generator = datagen.flow_from_directory(     
        train_dir,  # direktori data latih
        subset="training",
        target_size=(150, 150),# mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=3,
        # karena kita merupakan masalah klasifikasi 3 kelas maka menggunakan class_mode = 'categorical'
        class_mode="categorical")
 
validation_generator = datagen.flow_from_directory(
        validation_dir, # direktori data validasi
        subset="validation",
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=3, 
        # karena kita merupakan masalah klasifikasi 3 kelas maka menggunakan class_mode = 'categorical'
        class_mode="categorical")

Found 1533 images belonging to 3 classes.
Found 655 images belonging to 3 classes.


In [8]:
# Pembuatan model CNN

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax') #softmax = multiclass. sigmoid = 2 class. Dense = 1 itu berarti 1 class. yang di prediksi 3 class Jadi Dense(3, softmax)

])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [0]:
# model.compile(loss='binary_crossentropy',
#               optimizer=tf.optimizers.Adam(),
#               metrics=['accuracy'])

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
   optimizer=tf.keras.optimizers.SGD(),
   metrics=['accuracy'])

# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
# model.compile(loss=loss_fn, optimizer='adam')

In [0]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=20, mode='max') # https://keras.io/api/callbacks/early_stopping/

In [11]:
model.fit(
      train_generator,
      callbacks=[callback] , #callback early stop
      steps_per_epoch=25,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs=100,  # -> 100 kali propagasi balik
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=5,  # berapa batch yang akan dieksekusi pada setiap epoch
      verbose=2,)

Epoch 1/100
25/25 - 6s - loss: 1.1335 - accuracy: 0.3333 - val_loss: 1.0839 - val_accuracy: 0.3333
Epoch 2/100
25/25 - 5s - loss: 1.0930 - accuracy: 0.3200 - val_loss: 1.0562 - val_accuracy: 0.6000
Epoch 3/100
25/25 - 5s - loss: 1.0513 - accuracy: 0.5067 - val_loss: 1.0448 - val_accuracy: 0.6000
Epoch 4/100
25/25 - 6s - loss: 1.0075 - accuracy: 0.5733 - val_loss: 1.0317 - val_accuracy: 0.5333
Epoch 5/100
25/25 - 6s - loss: 0.8360 - accuracy: 0.6400 - val_loss: 0.8310 - val_accuracy: 0.6000
Epoch 6/100
25/25 - 6s - loss: 0.8732 - accuracy: 0.6533 - val_loss: 0.7149 - val_accuracy: 0.8000
Epoch 7/100
25/25 - 5s - loss: 0.8759 - accuracy: 0.5200 - val_loss: 0.7092 - val_accuracy: 0.6667
Epoch 8/100
25/25 - 5s - loss: 0.6366 - accuracy: 0.7333 - val_loss: 0.5285 - val_accuracy: 0.8000
Epoch 9/100
25/25 - 5s - loss: 0.7515 - accuracy: 0.6933 - val_loss: 0.8226 - val_accuracy: 0.6667
Epoch 10/100
25/25 - 5s - loss: 0.6808 - accuracy: 0.7733 - val_loss: 0.3439 - val_accuracy: 0.9333
Epoch 11/

In [12]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)

  class_lables = list(train_generator.class_indices.keys())

  print(classes)
  class_indice = 0
  for i in range(len(classes[0])):
      if classes[0][i] > classes[0][0]:
          class_indice = i
  test_class = class_lables[class_indice]
  print(test_class)


Using TensorFlow backend.


MessageError: ignored

# New Section